# Sistema de bicicletas compartidas

In [41]:
# Imports
import pandas as pd
import numpy as np

In [3]:
# install Pint if necessary

try:
    import pint
except ImportError:
    !pip install pint

In [ ]:
# download modsim.py if necessary

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/modsim.py')

In [ ]:
# import functions from modsim

from modsim import *

Vamos a ver un modelo sencillo de un sistema de bicicletas compartidas y muestrar las características de Python que usaremos para desarrollar simulaciones de sistemas del mundo real.

A lo largo del camino tomaremos decisiones que nos permitan mejorar gradualmente el modelo.

## Modelado de un sistema de bicicletas compartidas

Empezaremos con dos estaciones, `Robledo` y `c4ta`, y un número inicial de bicicletas en cada una. Usaremos diferentes librerias para representar el *estado* del sistema y actualizaremos ese estado a medida que las bicicletas se mueven.

In [10]:
state = dict(robledo=10, c4ta=2)
bikeshare_state = pd.Series(state, name="state")

Las asignaciones entre paréntesis crean dos variables, `Robledo` y `c4ta`, y les dan valores iniciales. Podemos mostrar estas variables individualmente, o inspeccionar el objeto completo para ver todas las variables de estado.

In [11]:
bikeshare_state.robledo

np.int64(10)

Y esto:

In [12]:
bikeshare_state.c4ta

np.int64(2)

In [13]:
print("el número de bicicletas en C4ta es:", bikeshare_state.c4ta)

el número de bicicletas en C4ta es: 2


O, para mostrar todas las variables de estado y sus valores, puedes escribir solo el nombre del objeto `State`.

In [14]:
bikeshare_state

robledo    10
c4ta        2
Name: state, dtype: int64

Estos valores conforman el *estado* del sistema.

Podemos definir una función `mostrar` que muestra el estado del sistema de bicicletas compartidas como una tabla.

In [17]:
def mostrar(obj):
    """Display a Series or Namespace as a DataFrame.

    Args:
        obj (object): Series or Namespace to display.

    Returns:
        pd.DataFrame: DataFrame representation of the object.
    """
    if isinstance(obj, pd.Series):
        df = pd.DataFrame(obj)
        return df
    elif hasattr(obj, "__dict__"):
        return pd.DataFrame(pd.Series(obj.__dict__), columns=["value"])
    else:
        return obj

In [18]:
mostrar(bikeshare_state)

,state
robledo,10
c4ta,2


No necesario usar 'mostrar', pero en algunos entormos suele verse mejor.

Podemos actualizar el estado asignando nuevos valores a las variables.

In [19]:
bikeshare_state.robledo = 9
bikeshare_state.c4ta = 3

O podemos usar *operadores de actualización*, `-=` y `+=`, para restar 1 de una estación y sumar 1 a la otra.

In [23]:
bikeshare_state.robledo -= 1
bikeshare_state.c4ta += 1

El resultado es el mismo de cualquier manera.

## Definición de funciones

Hasta ahora hemos utilizado funciones definidas en NumPy y en Pandas. Ahora vamos a definir nuestras propias funciones.

Cuando desarrollas código en Jupyter, a menudo es eficiente escribir unas pocas líneas, probarlas para confirmar que hacen lo que pretendes y luego usarlas para definir una nueva función.
Por ejemplo, estas líneas mueven una bicicleta de Robledo a C4ta:

In [24]:
bikeshare_state.robledo -= 1
bikeshare_state.c4ta += 1

En lugar de repetir estas líneas cada vez que se mueve una bicicleta, podemos definir una nueva función:

In [ ]:
def bicicleta_a_c4ta():
    bikeshare_state.robledo -= 1
    bikeshare_state.c4ta += 1

`def` es una palabra especial en Python que indica que estamos definiendo una nueva función. El nombre de la función es `bicicleta_a_c4ta`. Los paréntesis vacíos indican que esta función no necesita argumentos. Los dos puntos marcan el inicio de un *bloque de código* con sangría.

Las siguientes líneas son el *cuerpo* de la función; actualizan las variables `robledo` y `c4ta`.

In [29]:
bicicleta_a_c4ta()

Para *llamar* una función, escribe su nombre seguido de paréntesis. cuando no usas los parentesis pasa esto

In [33]:
bicicleta_a_c4ta


<function __main__.bicicleta_a_c4ta()>

## Print()

A medida que escribes programas más complejos, es fácil perder el hilo de lo que está ocurriendo.
Una de las herramientas más útiles para depurar es la instrucción print, que muestra texto en el cuaderno Jupyter.

Normalmente, cuando Jupyter ejecuta el código en una celda, muestra el valor de la última línea de código. Por ejemplo, si ejecutas:

In [35]:
bikeshare_state.robledo
bikeshare_state.c4ta

np.int64(7)

Jupyter ejecuta ambas líneas, pero solo muestra el valor de la segunda. Si quieres mostrar más de un valor, puedes usar instrucciones `print()`:

In [36]:
print(bikeshare_state.robledo)
print(bikeshare_state.c4ta)

5
7


Cuando llamas a `print`, puedes poner una variable entre paréntesis o proporcionar una secuencia de variables separadas por comas. Python busca los valores y los muestra en la misma línea, separados por un espacio.

In [37]:
print(bikeshare_state.robledo, bikeshare_state.c4ta)

5 7


Las sentencias `print` son útiles para depurar funciones. Por ejemplo, podemos añadir un `print` a `bicicleta_a_c4ta` para ver qué ocurre en cada llamada.

In [ ]:
def bicicleta_a_c4ta():
    print('Moviendo una bicicleta a C4TA')
    bikeshare_state.robledo -= 1
    bikeshare_state.c4ta += 1

Cada vez que llamamos a esta versión de la función, muestra un mensaje, lo que puede ayudarnos a seguir el rastro de lo que está haciendo el programa.
El mensaje en este ejemplo es una cadena (string), que es una secuencia de letras y otros símbolos entre comillas.

Al igual que "Bicicleta_a_robledo, podemos definir una función que mueva una bicicleta de C4ta a Robledo:

In [ ]:
def bicicleta_a_robledo():
    print('Moviendo una bicicleta a Robledo')
    bikeshare_state.robledo += 1
    bikeshare_state.c4ta -= 1

## If()

At this point we have functions that simulate moving bikes; now let's think about simulating customers. As a simple model of customer behavior, I will use a random number generator to determine when customers arrive at each station.

The ModSim library provides a function called `flip` that generates random "coin tosses".
When you call it, you provide a probability between 0 and 1, like this:

En este punto ya tenemos funciones que simulan el movimiento de bicicletas; ahora pensemos en simular a los clientes. Un modelo simple del comportamiento de los clientes puede ser un generador de números aleatorios para determinar cuándo llegan clientes a cada estación.

Creemos una función lanzamiento que genera “lanzamientos de moneda” aleatorios. Cuando la llamas, le proporcionas una probabilidad entre 0 y 1, así:

In [42]:
# this line sets the random number generator so the results in
# np.random.seed(17)

def lanzamiento(p=0.5):
    """Lanza una moneda con la probabilidad dada.

    Args:
        p (float): Probabilidad entre 0 y 1.

    Returns:
        bool: True or False.
    """
    return np.random.random() < p

In [43]:
lanzamiento(0.7)

False

El resultado es uno de dos valores: True con probabilidad 0.7 (en este ejemplo) o False con probabilidad 0.3. Si ejecutas flip de esta manera 100 veces, deberías obtener True unas 70 veces y False unas 30 veces. Pero los resultados son aleatorios, así que pueden diferir de estas expectativas.

True y False son valores especiales definidos por Python. Se llaman valores booleanos porque están relacionados con el álgebra booleana.

Ten en cuenta que no son cadenas de texto (strings). Hay una diferencia entre True, que es un valor booleano, y 'True', que es una cadena.

Podemos usar valores booleanos para controlar el comportamiento del programa, mediante una instrucción if:

Podemos entregar el resultado como cara o sello

In [ ]:
if lanzamiento(0.5):
    print('cara')
else:
    print('sello')

Esta funcion la podemos usar para mover bicicletas de una estación a otra, dependiendo del resultado del lanzamiento de la moneda. Por ejemplo, si el resultado es cara, movemos una bicicleta a C4TA; si es sello, movemos una bicicleta a Robledo.

In [ ]:
if lanzamiento(0.5):
    bicicleta_a_c4ta()

La proxima clase usaremos esto para realizar una simulación.